In [1]:
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from transformers import AutoModel, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
from datasets import Dataset
import torch

In [2]:
DEVICE = 0 if torch.cuda.is_available() else -1
INPUT_COL_NAME = 'text'
TARGET_COL_NAME = 'fake'
SMOKE = 100

In [3]:
tool_labels = {
    "generate": 1,
    "paraphrase": 1,
    "real": 0,
    "translate": 1,
    "unknown": 1
}

In [4]:
unknown_df = pd.read_csv("../detecting-generated-scientific-papers/fake_papers_train_part_public.csv")

In [5]:
unknown_df['type'] = unknown_df['fake'].apply(lambda x: 'unknown' if 1 else 'real' )
unknown_df['model'] = unknown_df['fake'].apply(lambda x: 'unknown' if 1 else 'real' )
unknown_df['tool'] = unknown_df['fake'].apply(lambda x: 'unknown' if 1 else 'real' )

In [6]:
test_df = unknown_df
test_df['pred'] = test_df['tool'].apply(lambda x: tool_labels[x])
if SMOKE:
    test_df = test_df[:SMOKE]

In [7]:
models = [
    { 
        "name": "multiclass",
        "model": "anon/deberta-v3-large-finetuned-synthetic-multi-class"
    },
]
datasets = ['all', 'generate', 'paraphrase', 'translate']

In [8]:
def get_predictions(model, texts):
    preds = []
    ds = Dataset.from_dict({
        "texts": list(texts.fillna(''))
    })
    for pred in tqdm(model(KeyDataset(ds, "texts"), batch_size=1), total=len(ds)):
        preds.append(
            pred
        )
    return preds

In [9]:
import numpy as np


def coverage_risk(confidences: np.ndarray, accuracies: np.ndarray):
    # From https://github.com/sleep3r/garrus/blob/a6fd1d44b06285918cefe54f421a004dc6f315cb/garrus/metrics/aurc.py
    sort_values = sorted(zip(confidences, accuracies), key=lambda x: x[0], reverse=True)
    sort_conf, sort_acc = zip(*sort_values)
    risk_list = []
    coverage_list = []
    risk = 0
    for i in range(len(sort_conf)):
        coverage = (i + 1) / len(sort_conf)
        coverage_list.append(coverage)

        if sort_acc[i] == 0:
            risk += 1

        risk_list.append(risk / (i + 1))
    return risk_list, coverage_list


def compute_area_under_risk_coverage_age(confidences: np.ndarray, accuracies: np.ndarray) -> float:
    """
    Area Under Risk-Coverage.
    $$ AURC (\kappa, f \mid V_{n}) = \frac{1}{n} \sum_{\theta \in \Theta} \hat{r} (f, g_{\theta} \mid V_{n}) $$
    From https://github.com/sleep3r/garrus/blob/a6fd1d44b06285918cefe54f421a004dc6f315cb/garrus/metrics/aurc.py
    """
    risk_list, coverage_list = coverage_risk(confidences, accuracies)
    risk_coverage_curve_area = 0
    for risk_value in risk_list:
        risk_coverage_curve_area += risk_value * (1 / len(risk_list))

    aurc = risk_coverage_curve_area
    return float(aurc), risk_list, coverage_list


In [10]:
label_map = {
    'LABEL_0': 1,
    'LABEL_1': 1,
    'LABEL_2': 0,
    'LABEL_3': 1
}

In [ ]:
results = []
for model in models:
    print(f"Getting results for {model['name']}")
    mdl = pipeline("text-classification", model['model'], device=DEVICE)
    for dataset in datasets:
        if dataset == 'all':
            df = test_df
        else:
            df = test_df.loc[
                (test_df['tool'] == dataset) | (test_df['tool'] == 'real')
            ]
        test_predictions = get_predictions(mdl, test_df[INPUT_COL_NAME])

        confidences = [
            pred['score']
            for pred in test_predictions
        ]
        predictions =  [
            label_map[pred['label']]
            for pred in test_predictions
        ]
        accuracies = [
            0 if prediction != y_true else 1 for (prediction, y_true)
            in zip(predictions, test_df["pred"])
        ]
        aurc, risk_list, coverage_list = compute_area_under_risk_coverage_age(confidences, accuracies)
        f1 = f1_score(test_df["pred"], predictions)
        precision = precision_score(test_df["pred"], predictions)
        recall = recall_score(test_df["pred"], predictions)

        acc = accuracy_score(test_df["pred"], predictions)
        data = {
            "model": model['model'],
            "name": model['name'],
            "aurc": aurc,
            "f1": f1,
           
            "precision": precision,
          
            "recall": recall,
     
            "acc": acc,
            "dataset": dataset,
            "risk_list": risk_list,
            "coverage_list": coverage_list,
            "label_distribution": list(zip([
                pred['label'] for pred in test_predictions
            ], accuracies))
        }
        results.append(data)

Getting results for multiclass


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
results

In [ ]:
pd.DataFrame(
    results
).to_csv('./multi_class_unknown_evaluation_results_smoke.csv')